# AS4501

# Bayesian inference

Francisco Förster

Bayes rule

## $\begin{align}
P(D , \theta| M) &= P(D | \theta, M) P(\theta | M) \\
&= P(\theta | D, M) P(D | M)
\end{align} $

This implies

## $P(\theta | D, M) = \frac{P(D | \theta, M) P(\theta | M)}{P(D | M)}$

The terms are called as follows:


* The **posterior**: $P(\theta | D, M)$

* The **likelihood**: $P(D | \theta, M)$

* The **prior**: $P(\theta | M)$

* The **evidence**: $P(D | M)$

The evidence is usually assumed to be some normalization constant, independent of $\theta$, but it has some applications.








### This means that we have a way of computing the probability of the model parameters given some model, the data and some prior information

### The posterior summarizes everything we know about the parameters given the data, including possible correlations between variables 

It is usually represented by a **corner plot** of the parameters $\theta$

Model example

![](spec.jpg)

Corner plot 

![](corner.png)

Another example 
![](corner2.png)

The **Maximum a Posteriori (MAP)** corresponds to the solution which maximizes the posterior.


![](MAP.png)



* Reporting posterior is the best we can do

* The problems start when we have too many dimensions, remember the curse of dimensionality

* Then, sampling the model parameters can become very expensive computationally

* To solve this problem, different **sampling** algorithms have been developed

## Use the cumulative distribution function (cdf)

![](cdf.png)

## Use rejection sampling

![](rejectionsampling.png)

### Basic idea:


1. Create a **proposal distribution $q(x)$** which satisfies $M q(x) \ge \tilde{p}(x)$, where M is some constant and $\tilde{p}(x)$ is the **target distribution**, some unnormalized version of $p(x)$. $M q(x)$ provides an upper envelope for $\tilde{p}$

2. Sample $x \sim q(x)$, which corresponds to picking a random $x$ location under the envelope, and then we sample $u \sim U(0, 1)$, whih corresponds to picking a random heigtht.

3. If $u > \frac{\tilde{p}(x)}{M q(x)}$, reject the sample, otherwise accept it

Let's define:

## $ S = \lbrace (x, u): u \le \tilde{p}(x)/M q(x) \rbrace$

and 

## $ S_0 = \lbrace (x, u): x \le x_0, u \le \tilde{p}(x) / M q(x) \rbrace$



The cdf of the accepted points is given by:

## $\begin{align}
P(x \le x_0 | x~\rm{accepted}) &= \frac{P(x \le x_0, x~\rm{accepted})}{P(x~\rm{accepted})} \\
 &= \frac{\int \int \mathbb{I}((x, u) \in S_0) q(x) du dx}{\int \int  \mathbb{I}((x, u) \in S) q(x) du dx} 
 = \frac{\int\limits_{-\inf}^{x_0} \tilde{p}(x)}{\int\limits_{-\inf}^{\inf} \tilde{p}(x)}
\end{align}$

Which is the cdf of $p(x)$




### Application to Bayesian statistics:

Suppose we want to draw unweighted samples from the posterior;

### $p(\theta | D) = p(D | \theta) p(\theta) / P(D)$.

We can use rejection sampling with the following target distribution:

### $\tilde{p}(\theta) = p(D | \theta) p(\theta)$,

the following proposal distribution:

### $q(\theta) = p(\theta)$

and 

### $M = p(D | \hat{\theta})$,

where $\hat{\theta} = arg \max(p(D | \theta))$,

the maximum likelihood estimator of $\theta$.

Then, we accept points with probability

### $\frac{\tilde{p}(\theta)}{M q(\theta)} = \frac{p(D|\theta)}{p(D | \hat{\theta})}$

Thus, samples from the prior which have high likelihood are more likely to be retained in the posterior. 

However, if there is a big mismatch between prior and posterior this procedure is very inefficient. This is usually the case in high dimensions!

## Markov Chain Monte Carlo (MCMC) sampling

* Family of methods to sample distributions in high dimensions

* Among top 10 most important algorithms of the century!

* **Basic idea**:

    * Construct a Markov chain (sequence of values where the next value $i+1$ depends only on the present value $i$) on the state space whose stationary distribution is the target density $p^*(x)$
    
    * That is, we perform a random walk on the state space in such a way that the fraction of time we spend in each state $x$ is proportional to $p^*(x)$.
    
    * By drawing correlated samples $x_0, x_1, x_2$ from the chain, we can perform Monte Carlo integration w.r.t. $p^*$

## Gibbs sampling

* One of the most popular MCMC algorithm

* **Basic  idea**:
    
    * we sample each variable in turn, conditioned on the values of all the other variables in the distribution
        
    * given a joint sample $x^s$ of all the variables, we generate a new sample $x^{s+1}$ by sampling each component in turn, based on the most recent values of the variables, e.g. for 3 dimensions:
    
        * $x_1^{s+1} \sim p(x_1 | x_2^s, x_3^s)$
        * $x_2^{s+1} \sim p(x_2 | x_1^s, x_3^s)$
        * $x_3^{s+1} \sim p(x_3 | x_1^s, x_2^s)$
        
    * The expression $p(x_i | x_{-i})$ is called the **full conditional** for variable $i$
    
    * It is usually necessary to discard some of the initial samples until the Markov Chain has **burned in**.






![](gibbs.jpg)

## Metropolis-Hastings algorithm

* The Models where we can compute the full conditional probability are limited

* Even when this exists, Gibbs sampling can be quite slow

* More general algorithm is the **Metropolis Hastings (MH)** algorithm.

* **Basic idea**:

    * at each step, we propose to move from the current state $x$ to a new state $x'$ with probability $q(x'|x)$, where $q$ is called the **proposal distribution** or **kernel**.
    
    * the user is free to use any kind of proposal they want, subject to some conditions
    
    * Commonly used proposal: symmetric Gaussian distribution centered on the current state: $q(x'|x) = N(x'|x, \Sigma)$; which is called a **random walk MH**
    
    * if we use a proposal of the form $q(x'|x) = q(x')$, the new state is independent of the old state, and the method is known as **independence sampler**.
    
    * having proposed $x'$, we then decide to accept or reject it according to some formula, which ensures that the fraction of time spent in each state is proportional to $p^*(x)$
    
    * if the proposal is accepted, the new state is $x'$, otherwise the state remains the same.
    
    * if the proposal is symmetric, i.e. $q(x'|x) = q(x | x')$ the acceptance probability is given by the following formula:
    
        * $r = \min (1, \frac{p^*(x')}{p^*(x)})$
        
        * i.e. we always move to $x'$ if it is more probable than $x$, otherwise we move there with some probability.
        
    * **It can be shown that this procedure ensures that the fraction of time we spend in each state $x$ is proportional to $p^*(x)$.**
    
    * Note that if the proposal is asymmetric, so $q(x'|x) \ne q(x|x')$, we need the **Hasting correction**:
    
        * $r = \min(1, \alpha)$
        
        * $\alpha = \frac{p^*(x') q(x|x')}{p^*(x) q(x'|x)} = \frac{p^*(x')/q(x'|x)}{p^*(x) / q(x|x')}$
        
* An important reason why MH is useful is that, when evaluating for $\alpha$, we only need to know the target density up to a normalization constant (which gets canceled in the formula above)
      




![](MH.png)

![](3dRosenbrock.png)

## PyMC https://docs.pymc.io/

![](MCMC.png)

![](MCMC2.png)

![](MCMC3.png)

![](MCMC4.png)

# Affine invariant MCMC

Method popularized by the **emcee** code http://dfm.io/emcee/current/

Paper from Goodman & Weare in here: https://msp.org/camcos/2010/5-1/camcos-v5-n1-p04-p.pdf


**Basic idea**:
    
* The method has many **parallel walkers**, each sampling from the distribution

* the walkers are only allowed to do linear combinations of their solutions, which simplifies significantly the theoretical derivation (**stretch move**)

* usually much faster than MH

* Proposal of the form $x_k(t) \rightarrow Y = x_j + z(x_k(t) - x_j)$

* If Z satisfies the symmetry condition: $g(\frac{1}{z}) = z g(z)$, then the move is symmetric in the sense that: $p(x_k(t) \rightarrow Y) = p(y \rightarrow x_k(t))$

![](stretch.png)

* the particular distribution used is the  following:

## $g(z)=  \begin{cases} \frac{1}{\sqrt{z}} & \text{if} z \in [\frac{1}{a}, a] \\ 0, & \text{otherwise} \end{cases}$

where the parameter $a > 1$ can be adjusted to improve performance


![](stretch2.png)

Example using SN data from the HiTS survey (Förster et al. 2016) fitted to hydrodynamical models from Moriya et al. 2018

![](evol15X.png)

![](corner15X.png)

![](models15X.png)